In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
import itertools
from tabulate import tabulate
import os

In [2]:
df = pd.read_csv('/home/sourav/major/Updated_Packet_Trace2.csv', encoding='latin-1')
df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,APP_LAYER_ARRIVAL_TIME(ÂµS),TRX_LAYER_ARRIVAL_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
1,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
2,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
3,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
4,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,Control_Packet,WLAN_ACK,NODE-17,NODE-1,NODE-17,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,-,-,-,Normal
80697,4434,0.0,CBR,App1_CBR,NODE-16,NODE-10,NODE-1,NODE-17,88660000.0,88660000.0,...,NaN,NaN,NaN,NaN,NaN,192.168.0.17,192.168.0.11,192.168.0.2,192.168.0.18,Normal
80698,0,NaN,Control_Packet,WLAN_ACK,NODE-17,NODE-1,NODE-17,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,-,-,-,Normal
80699,4432,0.0,CBR,App1_CBR,NODE-16,NODE-10,NODE-17,NODE-10,88620000.0,88620000.0,...,NaN,NaN,NaN,NaN,NaN,192.168.0.17,192.168.0.11,192.168.0.18,192.168.0.11,Normal


In [3]:
from sklearn.preprocessing import LabelEncoder
def le(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

le(df)
df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,APP_LAYER_ARRIVAL_TIME(ÂµS),TRX_LAYER_ARRIVAL_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,1,8,20,0,20,0,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
1,0,NaN,1,8,20,0,20,11,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
2,0,NaN,1,8,20,0,20,14,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
3,0,NaN,1,8,20,0,20,15,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
4,0,NaN,1,8,20,0,20,16,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,1,12,8,1,8,0,NaN,NaN,...,NaN,2,2,1,NaN,0,0,0,0,1
80697,4434,0.0,0,2,7,2,0,8,88660000.0,88660000.0,...,NaN,2,2,1,NaN,8,2,11,9,1
80698,0,NaN,1,12,8,1,8,0,NaN,NaN,...,NaN,2,2,1,NaN,0,0,0,0,1
80699,4432,0.0,0,2,7,2,8,1,88620000.0,88620000.0,...,NaN,2,2,1,NaN,8,2,9,2,1


In [7]:
df.columns = df.columns.str.strip()
# df = df.drop([ 'REMOTE_ADDRESS'], axis=1)
# df = df.drop([ 'LOCAL_ADDRESS'], axis=1)
# df = df.drop([ 'MAC_LAYER_ARRIVAL_TIME(µS)'], axis=1)
#df = df.drop([ 'NW_LAYER_ARRIVAL_TIME(µS)'], axis=1)
df = df.drop([ 'APP_LAYER_ARRIVAL_TIME(ÂµS)'], axis=1)
df = df.drop([ 'NW_LAYER_ARRIVAL_TIME(ÂµS)'], axis=1)

df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,PHY_LAYER_ARRIVAL_TIME(ÂµS),PHY_LAYER_START_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,1,8,20,0,20,0,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
1,0,NaN,1,8,20,0,20,11,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
2,0,NaN,1,8,20,0,20,14,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
3,0,NaN,1,8,20,0,20,15,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
4,0,NaN,1,8,20,0,20,16,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,1,12,8,1,8,0,99893651.00,99893806.00,...,NaN,2,2,1,NaN,0,0,0,0,1
80697,4434,0.0,0,2,7,2,0,8,99894059.11,99895054.11,...,NaN,2,2,1,NaN,8,2,11,9,1
80698,0,NaN,1,12,8,1,8,0,99895118.12,99895273.12,...,NaN,2,2,1,NaN,0,0,0,0,1
80699,4432,0.0,0,2,7,2,8,1,99895421.23,99898246.23,...,NaN,2,2,1,NaN,8,2,9,2,1


In [8]:
X = df.drop('Malicious/Normal', axis=1)
y = df['Malicious/Normal']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y

0        0
1        0
2        0
3        0
4        0
        ..
80696    1
80697    1
80698    1
80699    1
80700    1
Name: Malicious/Normal, Length: 80701, dtype: int64

In [10]:
rfc = RandomForestClassifier()

rfe = RFE(rfc, n_features_to_select=10)
rfe = rfe.fit(X_train, y_train)

feature_map = [(i, v) for i, v in itertools.zip_longest(rfe.get_support(), X_train.columns)]
selected_features = [v for i, v in feature_map if i==True]

selected_features

['SOURCE_ID',
 'DESTINATION_ID',
 'TRANSMITTER_ID',
 'RECEIVER_ID',
 'PHY_LAYER_ARRIVAL_TIME(ÂµS)',
 'PHY_LAYER_START_TIME(ÂµS)',
 'PHY_LAYER_END_TIME(ÂµS)',
 'MAC_LAYER_PAYLOAD(Bytes)',
 'PACKET_STATUS',
 'SOURCE_IP']

In [11]:
X_train = X_train[selected_features]
# X_train
X_test=X_test[selected_features]
X_test

,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,PHY_LAYER_ARRIVAL_TIME(ÂµS),PHY_LAYER_START_TIME(ÂµS),PHY_LAYER_END_TIME(ÂµS),MAC_LAYER_PAYLOAD(Bytes),PACKET_STATUS,SOURCE_IP
23601,6,12,6,11,12986655.80,12986842.80,12986842.81,20,2,7
62322,7,2,7,14,58240872.62,58241865.62,58241865.63,1500,0,8
23307,19,14,19,13,12932629.25,12932762.25,12932762.26,48,0,21
16370,12,9,9,8,6753195.01,6753460.01,6753460.02,64,0,13
379,18,0,18,8,106736.22,107200.22,107200.23,32,0,20
...,...,...,...,...,...,...,...,...,...,...
54073,12,16,12,15,48402684.03,48402824.03,48402824.04,60,2,13
39691,11,7,11,6,30821762.24,30821917.24,30821917.25,0,2,0
71878,7,2,7,17,75381191.23,75383335.23,75383335.24,1508,0,8
59348,11,7,11,8,55440833.89,55440982.89,55440982.90,76,2,15


In [12]:
lg_model = LogisticRegression(random_state = 42)
lg_model.fit(X_train, y_train)

/home/sourav/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [13]:
lg_train, lg_test = lg_model.score(X_train , y_train), lg_model.score(X_test , y_test)

print(f"Training Score: {lg_train}")
print(f"Test Score: {lg_test}")

Training Score: 0.704182156133829
Test Score: 0.7100551390867976


In [16]:
custom_row = X_test.iloc[[12346]] 
prediction = lg_model.predict(custom_row)

In [17]:
print('Predicted Target Variable:', prediction[0])

Predicted Target Variable: 1


In [18]:
SEED = 42
lr = LogisticRegression()

In [19]:
from sklearn.model_selection import cross_val_score
models = {}
models['logisticregression']= lr

In [20]:
scores = {}
for name in models:
  scores[name]={}
  for scorer in ['precision','recall']:
    scores[name][scorer] = cross_val_score(models[name], X_train, y_train, cv=10, scoring=scorer)

/home/sourav/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sourav/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

In [21]:
def line(name):
  return '*'*(25-len(name)//2)

for name in models:
  print(line(name), name, 'Model Validation', line(name))

  for scorer in ['precision','recall']:
    mean = round(np.mean(scores[name][scorer])*100,2)
    stdev = round(np.std(scores[name][scorer])*100,2)
    print ("Mean {}:".format(scorer),"\n", mean,"%", "+-",stdev)
    print()

**************** logisticregression Model Validation ****************
Mean precision: 
 70.41 % +- 0.02

Mean recall: 
 99.97 % +- 0.1



In [22]:
preds={}
for name in models:
    models[name].fit(X_train, y_train)
    preds[name] = models[name].predict(X_test)
print("Predictions complete.")

Predictions complete.


/home/sourav/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
def line(name,sym="*"):
    return sym*(25-len(name)//2)
target_names=["normal","anamoly"]
for name in models:
    print(line(name), name, 'Model Testing', line(name))
    print(confusion_matrix(y_test, preds[name]))
    print(line(name,'-'))
    print(classification_report(y_test, preds[name], target_names=target_names))

**************** logisticregression Model Testing ****************
[[    0  4680]
 [    0 11461]]
----------------
              precision    recall  f1-score   support

      normal       0.00      0.00      0.00      4680
     anamoly       0.71      1.00      0.83     11461

    accuracy                           0.71     16141
   macro avg       0.36      0.50      0.42     16141
weighted avg       0.50      0.71      0.59     16141



/home/sourav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sourav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sourav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
